In [53]:
import os
import PIL
import tensorflow as tf
import io
import itertools
import numpy as np
import sklearn.metrics
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense , Dropout

In [54]:
batch_size = 16
img_height = 128
img_width = 128

In [55]:
train_ds = tf.keras.utils.image_dataset_from_directory(
 'data2/train',
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  color_mode='grayscale')

Found 1288 files belonging to 8 classes.
Using 1031 files for training.


In [56]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  'data2/train',
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  color_mode='grayscale')

Found 1288 files belonging to 8 classes.
Using 257 files for validation.


In [57]:
class_names = train_ds.class_names
print(class_names)

['Book', 'Comb', 'Cup', 'Glasses', 'Mouse', 'Pen', 'Remote', 'Scissors']


In [58]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(16, 128, 128, 1)
(16,)


In [59]:
for images, labels in val_ds:
    images_val = images.numpy()
    labels_val = labels.numpy()

In [60]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [61]:
normalization_layer = layers.Rescaling(1./255)

In [62]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 1.0


In [63]:
num_classes = len(class_names)

model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 1)),
  layers.Conv2D(100, 5, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(100, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [64]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [65]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_5 (Rescaling)     (None, 128, 128, 1)       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 128, 128, 100)     2600      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 64, 64, 100)      0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 64, 64, 100)       90100     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 32, 32, 100)      0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 32, 32, 100)      

In [66]:
def plot_confusion_matrix(cm, class_names):
    """
    Returns a matplotlib figure containing the plotted confusion matrix.

    Args:
    cm (array, shape = [n, n]): a confusion matrix of integer classes
    class_names (array, shape = [n]): String names of the integer classes
    """
    figure = plt.figure(figsize=(12, 12))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    # Normalize the confusion matrix.
    cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

    # Use white text if squares are dark; otherwise black.
    threshold = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
    return figure

In [67]:
def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    
    # Closing the figure prevents it from being displayed directly inside the notebook.
    plt.close(figure)
    
    buf.seek(0)
    
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    
    return image

In [68]:
# Define a file writer variable for logging purposes
file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')

def log_confusion_matrix(epoch, logs):
    # Use the model to predict the values from the validation dataset.
    test_pred_raw = model.predict(images_val)
    test_pred = np.argmax(test_pred_raw, axis=1)

    # Calculate the confusion matrix.
    cm = sklearn.metrics.confusion_matrix(labels_val, test_pred)
    
    # Log the confusion matrix as an image summary.
    figure = plot_confusion_matrix(cm, class_names=['Book', 'Comb', 'Cup', 'Glasses', 'Mouse', 'Pen', 'Remote', 'Scissors'])
    cm_image = plot_to_image(figure)

    # Log the confusion matrix as an image summary.
    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [69]:
# Defining the callbacks
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=0)

In [70]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    mode = 'auto',
    min_delta = 0,
    patience = 2,
    verbose = 0, 
    restore_best_weights = True
)

In [71]:
epochs=20
model.fit(
  train_ds,
  validation_data=val_ds,
  callbacks = [tensorboard_callback, cm_callback,early_stopping],
  epochs=epochs
)

Epoch 1/20
65/65 [==============================] - 62s 926ms/step - loss: 1.3764 - accuracy: 0.5664 - val_loss: 0.2608 - val_accuracy: 0.9416
Epoch 2/20
65/65 [==============================] - 60s 922ms/step - loss: 0.1322 - accuracy: 0.9641 - val_loss: 0.2029 - val_accuracy: 0.9728
Epoch 3/20
65/65 [==============================] - 67s 1s/step - loss: 0.0451 - accuracy: 0.9855 - val_loss: 0.1026 - val_accuracy: 0.9767
Epoch 4/20
65/65 [==============================] - 55s 847ms/step - loss: 0.0465 - accuracy: 0.9952 - val_loss: 0.1067 - val_accuracy: 0.9844
Epoch 5/20
65/65 [==============================] - 63s 971ms/step - loss: 0.0232 - accuracy: 0.9913 - val_loss: 0.0922 - val_accuracy: 0.9883
Epoch 6/20
65/65 [==============================] - 59s 902ms/step - loss: 0.0245 - accuracy: 0.9942 - val_loss: 0.1219 - val_accuracy: 0.9883
Epoch 7/20
65/65 [==============================] - 59s 912ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.1187 - val_accuracy: 0.9883


In [72]:
model_json = model.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')
model.save_weights('model-bw.h5')
print('Weights saved')


Model Saved
Weights saved


In [73]:
test_ds = tf.keras.utils.image_dataset_from_directory(
  'data2/test',
  image_size=(img_height, img_width),
  batch_size=batch_size,
  color_mode='grayscale')


Found 328 files belonging to 8 classes.


In [74]:
class_names = test_ds.class_names
print(class_names)

['Book', 'Comb', 'Cup', 'Glasses', 'Mouse', 'Pen', 'Remote', 'Scissors']


In [75]:
model.evaluate(test_ds)

21/21 [==============================] - 6s 183ms/step - loss: 2.3188 - accuracy: 0.7012


[2.3187615871429443, 0.7012194991111755]

In [143]:
import cv2
import numpy as np
img = tf.keras.utils.load_img(
    "./data/test/Comb/4.jpg", target_size=(img_height, img_width,),grayscale=True
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

This image most likely belongs to Mouse with a 57.95 percent confidence.
